In [10]:
import netCDF4 as nc
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr
from IO_functions import *
from parameters import *
from statsFunc import crossValidateKfold
import scipy.io


Load data

In [2]:
tas1=year_load(CORDEX_path,mohc_dates,most_dates,years,'tas','ICHEC-EC-EARTH','rcp85','r12i1p1','SMHI-RCA4','v1','2006').expand_dims({'obs':1})
tas2=month_load("C:/Users/predr/Desktop/",mohc_dates,most_dates,years,'tas','ICHEC-EC-EARTH','rcp85','r12i1p1','SMHI-RCA4','v1','2008','12')
hurs=month_load("C:/Users/predr/Desktop/",mohc_dates,most_dates,years,'hurs','CNRM-CERFACS-CNRM-CM5','rcp45','r1i1p1','SMHI-RCA4','v1','2008','12')

AttributeError: 'list' object has no attribute 'expand_dims'

In [7]:
tas1['tas']

<xarray.DataArray 'tas' (obs: 1, time: 365, lat: 90, lon: 134)>
array([[[[291.60153, 291.72653, 291.78903, ..., 279.32028, 278.35153,
          278.35153],
         [291.41403, 291.50778, 291.57028, ..., 278.60153, 278.44528,
          278.72653],
         [291.25778, 291.35153, 291.38278, ..., 279.60153, 279.88278,
          279.82028],
         ...,
         [244.66403, 244.88278, 245.47653, ..., 272.91403, 272.88278,
          272.66403],
         [245.47653, 245.72653, 245.97653, ..., 273.32028, 273.32028,
          273.10153],
         [245.97653, 246.53903, 246.85153, ..., 273.50778, 273.38278,
          273.32028]],

        [[291.85492, 291.82367, 291.85492, ..., 278.7143 , 278.55804,
          278.6518 ],
         [291.76117, 291.82367, 291.85492, ..., 278.87054, 278.80804,
          278.66742],
         [291.60492, 291.66742, 291.7143 , ..., 278.85492, 279.3393 ,
          280.7768 ],
...
         [263.2449 , 264.1824 , 265.9324 , ..., 271.40115, 271.33865,
          271.40115],
         [265.9324 , 266.6824 , 267.3074 , ..., 271.02615, 270.96365,
          271.02615],
         [267.3074 , 268.3699 , 268.71365, ..., 270.83865, 270.9324 ,
          270.96365]],

        [[290.61005, 290.61005, 290.5788 , ..., 273.54755, 273.3913 ,
          273.36005],
         [290.61005, 290.61005, 290.61005, ..., 273.31317, 273.40692,
          273.43817],
         [290.40692, 290.4538 , 290.53192, ..., 273.67255, 274.25067,
          275.25067],
         ...,
         [262.03192, 262.43817, 263.34442, ..., 268.37567, 268.23505,
          268.12567],
         [263.34442, 263.87567, 264.42255, ..., 268.17255, 267.98505,
          267.81317],
         [264.42255, 265.31317, 265.5788 , ..., 268.23505, 268.0788 ,
          267.98505]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2006-01-01T12:00:00 ... 2006-12-31T12:00:00
  * lon      (lon) float64 -22.0 -21.5 -21.0 -20.5 -20.0 ... 43.0 43.5 44.0 44.5
  * lat      (lat) float64 27.0 27.5 28.0 28.5 29.0 ... 69.5 70.0 70.5 71.0 71.5
    height   float64 ...
Dimensions without coordinates: obs
Attributes:
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    units:          K
    cell_methods:   time: mean

In [12]:
year_begin=2006
year_end=2010
data=xr.open_dataset("C:/Users/predr/Desktop/2081-2085_test_output.nc")
with xr.open_dataset("C:/Users/predr/Desktop/merged_monthly.nc") as monthly_mask:
    # Convert adjr2 and rmse data to xarray.Dataset format
        ds = xr.Dataset(
            data_vars=dict(
                adjr2=(["time", "lat", "lon"], data["adjr2"]),
                rmse=(["time", "lat", "lon"], data["rmse"]),
                time_bnds=(["time", "bnds"], monthly_mask['time_bnds'][(year_begin-2006)*12:(year_end+1-2006)*12])
            ),
            coords=dict(
                lon=(['lon'], data['lon']),
                lat=(["lat"], data['lat']),
                time=monthly_mask['time'][(year_begin-2006)*12:(year_end+1-2006)*12],
            ),
            attrs=dict(description="Monthly Adj. R^2 and RMSE results for tas/hurs relationships."))

        # Save to NCDF4
        ds.to_netcdf(str(year_begin)+ '-' + str(year_end) + "_test_output.nc")

In [4]:
data = xr.open_dataset("C:/Users/predr/Desktop/2061-2065_test_output.nc")
data

<xarray.Dataset>
Dimensions:  (lat: 90, lon: 134, time: 60)
Coordinates:
  * lon      (lon) float64 -22.0 -21.5 -21.0 -20.5 -20.0 ... 43.0 43.5 44.0 44.5
  * lat      (lat) float64 27.0 27.5 28.0 28.5 29.0 ... 69.5 70.0 70.5 71.0 71.5
    height   float64 ...
  * time     (time) datetime64[ns] 2061-01-16T12:00:00 ... 2065-12-16T12:00:00
Data variables:
    adjr2    (time, lat, lon) float64 ...
    rmse     (time, lat, lon) float64 ...
Attributes:
    description:  Monthly Adj. R^2 and RMSE results for tas/hurs relationships.

In [6]:
mat = scipy.io.loadmat('G:\My Drive\CLIMRISK_Pedja\CLIMRISK_RCP2.6_SSP1_IIASA_50pctl_50climsens.mat')
#plt.pcolor(tas_climrisk[:,:,1])
mat['TEMPERATURES_FINAL'][100,1,:]


IndexError: index 100 is out of bounds for axis 0 with size 90

Load temperature patterns:

In [26]:
data = xr.open_dataset('G:/My Drive/CLIMRISK_Pedja/remapped_tas_patterns_EUR-11_NCC-NorESM1-M_rcp26_r1i1p1_SMHI-RCA4_v1_day_20960101-21001231.nc')
data.sel(time='2096')['tas']

<xarray.DataArray 'tas' (time: 365, lat: 90, lon: 134)>
[4401900 values with dtype=float32]
Coordinates:
  * time     (time) object 2096-01-01 12:00:00 ... 2096-12-31 12:00:00
  * lon      (lon) float64 -22.0 -21.5 -21.0 -20.5 -20.0 ... 43.0 43.5 44.0 44.5
  * lat      (lat) float64 27.0 27.5 28.0 28.5 29.0 ... 69.5 70.0 70.5 71.0 71.5
    height   float64 ...
Attributes:
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    units:          K
    cell_methods:   time: mean

In [31]:
tas1=tas1.expand_dims({'obs':1})
tas2=tas2.expand_dims({'obs':1})

ValueError: Dimension obs already exists.

In [32]:
tas1['time']

<xarray.DataArray 'time' (time: 31)>
array(['2008-12-01T12:00:00.000000000', '2008-12-02T12:00:00.000000000',
       '2008-12-03T12:00:00.000000000', '2008-12-04T12:00:00.000000000',
       '2008-12-05T12:00:00.000000000', '2008-12-06T12:00:00.000000000',
       '2008-12-07T12:00:00.000000000', '2008-12-08T12:00:00.000000000',
       '2008-12-09T12:00:00.000000000', '2008-12-10T12:00:00.000000000',
       '2008-12-11T12:00:00.000000000', '2008-12-12T12:00:00.000000000',
       '2008-12-13T12:00:00.000000000', '2008-12-14T12:00:00.000000000',
       '2008-12-15T12:00:00.000000000', '2008-12-16T12:00:00.000000000',
       '2008-12-17T12:00:00.000000000', '2008-12-18T12:00:00.000000000',
       '2008-12-19T12:00:00.000000000', '2008-12-20T12:00:00.000000000',
       '2008-12-21T12:00:00.000000000', '2008-12-22T12:00:00.000000000',
       '2008-12-23T12:00:00.000000000', '2008-12-24T12:00:00.000000000',
       '2008-12-25T12:00:00.000000000', '2008-12-26T12:00:00.000000000',
       '2008-12-27T12:00:00.000000000', '2008-12-28T12:00:00.000000000',
       '2008-12-29T12:00:00.000000000', '2008-12-30T12:00:00.000000000',
       '2008-12-31T12:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2008-12-01T12:00:00 ... 2008-12-31T12:00:00
    height   float64 2.0
Attributes:
    standard_name:  time
    long_name:      time
    bounds:         time_bnds
    axis:           T

In [34]:
x['tas'][time,lat,lon]

NameError: name 'time' is not defined

In [41]:
x=[]
x=xr.concat([tas1, tas2], dim="obs")
#x['obs'].shape
x['tas'][:,30,89,133]

<xarray.DataArray 'tas' (obs: 2)>
array([269.7248 , 271.09338], dtype=float32)
Coordinates:
    time     datetime64[ns] 2008-12-31T12:00:00
    lon      float64 44.5
    lat      float64 71.5
    height   float64 2.0
Dimensions without coordinates: obs
Attributes:
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    units:          K
    cell_methods:   time: mean

In [3]:
coef=np.zeros((90,134,2))
adjr2=np.zeros((90,134))
rmse=np.zeros((90,134))

for lat in range(90):
    for lon in range(134):
        tas_cell_month=tas1['tas'][:,lat,lon]
        hurs_cell_month=hurs['hurs'][:,lat,lon]
        print(np.concatenate([tas_cell_month,tas_cell_month]))

 280.60632 280.14078
 280.42892 281.0238  280.83966 280.88663 282.8173  282.3526  280.3328
 278.44604 277.4474  277.00903 278.4623  279.40125 278.6979  278.0526
 279.1963  276.91974 278.9837  280.00363 276.96158 278.3578  282.8227
 279.32956 274.97012 276.36542 280.85562 280.14798 280.65198 279.74518
 280.2401  280.60632 280.14078 280.42892 281.0238  280.83966 280.88663
 282.8173  282.3526  280.3328  278.44604 277.4474  277.00903 278.4623
 279.40125 278.6979  278.0526  279.1963  276.91974 278.9837  280.00363
 276.96158 278.3578  282.8227  279.32956 274.97012 276.36542]
[280.94937 280.94485 281.1051  280.4483  280.94324 281.20007 281.01578
 281.21017 281.72693 282.1209  281.0585  283.47354 282.43073 279.92654
 278.00854 277.1349  276.85278 278.8998  279.72937 277.4479  277.5526
 279.47754 277.0135  279.45245 280.26926 276.64908 277.77966 282.72894
 280.06393 275.87637 276.6623  280.94937 280.94485 281.1051  280.4483
 280.94324 281.20007 281.01578 281.21017 281.72693 282.1209  281.0585
 

KeyboardInterrupt: 

In [4]:
tas_all_month

NameError: name 'tas_all_month' is not defined

In [59]:
CORDEX_path="C:/Users/predr/Desktop/"
# Parameters
folds= 10
coef=np.zeros((60,90,134,2))
adjr2=np.zeros((60,90,134))
rmse=np.zeros((60,90,134))
month_count=0 #start counting months

for year in range(2006,2010):
    for month in range(1,13):
        tas_all_month=[]
        hurs_all_month=[]
        for gcm in gcms:
            for rcp in rcps:
                for rp in rps:
                    for rcm in rcms:
                        for version in versions:
                            # Load data
                            try:
                                tas=month_load(CORDEX_path,mohc_dates,most_dates,years,'tas',gcm,rcp,rp,rcm,version,str(year),str(month))
                                hurs=month_load(CORDEX_path,mohc_dates,most_dates,years,'hurs',gcm,rcp,rp,rcm,version,str(year),str(month))
                                # Concatenate
                                if any(tas_all_month):
                                    tas_all_month=xr.concat([tas_all_month, tas], dim="time")
                                else:
                                    tas_all_month=tas
                                if any(hurs_all_month):
                                    hurs_all_month=xr.concat([hurs_all_month, hurs], dim="time")
                                else:
                                    hurs_all_month=hurs
                            except Exception as e: 
                                print(e)
        # Do stuff with monthly data here
        # Regression
        # Might have to do per cell
        #print(tas_all_month.sizes)
        for lat in range(1):
            for lon in range(5):
                tas_cell_month=tas_all_month['tas'][:,lat,lon]
                hurs_cell_month=hurs_all_month['hurs'][:,lat,lon]
                try:
                    coef[month,lat,lon,:],adjr2[month,lat,lon],rmse[month,lat,lon]=crossValidateKfold(tas_cell_month,hurs_cell_month,folds)  
                    print(str(month) + '/' + str(year) + ' finished')     
                except Exception as e: 
                    print(e)
                    pass
        month_count=month_count+1 # increment month

                #print(str(month) + '/' + str(year) + ' finished')


list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
4/2009 finished
4/2009 finished
4/2009 finished
4/2009 finished
4/2009 finished
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' ob

In [5]:
monthly_mask = xr.open_dataset("C:/Users/predr/Desktop/CNRM-CERFACS-CNRM-CM5/merged_monthly_time.nc")
monthly_mask['time'][2:1097]

<xarray.DataArray 'time' (time: 1095)>
array(['2006-03-16T12:00:00.000000000', '2006-04-16T00:00:00.000000000',
       '2006-05-16T12:00:00.000000000', ..., '2097-03-16T12:00:00.000000000',
       '2097-04-16T00:00:00.000000000', '2097-05-16T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2006-03-16T12:00:00 ... 2097-05-16T12:00:00
    height   float64 ...
Attributes:
    standard_name:  time
    long_name:      time
    bounds:         time_bnds
    axis:           T

In [55]:
monthly_mask['time'].to_netcdf("C:/Users/predr/Desktop/CNRM-CERFACS-CNRM-CM5/merged_monthly_time.nc")

In [57]:
foo = xr.DataArray(adjr2, coords=[monthly_mask['time'], hurs_all_month['lat'],hurs_all_month['lon']], dims=["time", "lat","lon"])

ValueError: conflicting sizes for dimension 'time': length 60 on the data but length 1140 on coordinate 'time'

In [48]:
ds = xr.Dataset(
    data_vars=dict(
        adjr2=(["time", "lat", "lon"], adjr2),
        rmse=(["time", "lat", "lon"], rmse),
    ),
    coords=dict(
        lon=(['lon'], hurs_all_month['lon']),
        lat=(["lat"], hurs_all_month['lat']),
        time=monthly_mask['time'],
    ),
    attrs=dict(description="Weather related data."))

In [50]:
ds.to_netcdf("C:/Users/predr/Desktop/CNRM-CERFACS-CNRM-CM5/test_output.nc")

In [66]:
range(10,11)[0]

10

In [1]:

import netCDF4 as nc
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr
from IO_functions import *
from parameters import *
#import statsmodels.api as sm
from sklearn import neighbors
#from statsFunc import crossValidateKfold
from pathCORDEX import *
import time
import sys

year_begin=2006
year_end=2100

for year in range(year_begin,year_end+1):
    tas_all_year=[]
    for gcm in gcms:
        for rcp in rcps:
            for rp in rps:
                for rcm in rcms:
                    for version in versions:
                        # Load data
                        try:
                            tas=year_load(CORDEX_path,mohc_dates,most_dates,years,'tas',gcm,rcp,rp,rcm,version,str(year)).expand_dims({'obs':1}) # Expand the dimension for observations
                            # Concatenate
                            if any(tas_all_year):
                                tas_all_year=xr.concat([tas_all_year, tas], dim="obs")
                            else:
                                tas_all_year=tas
                        except Exception as e:
                        #print('K-fold loop: ' + str(e))
                            pass

tas_all_year.to_dataframe.to_netcdf(str(year_begin)+ '-' + str(year_end) + "_test_output.nc")


AttributeError: 'list' object has no attribute 'to_dataframe'

In [13]:
year_begin=2006
year_end=2010
tas=year_load(CORDEX_path,mohc_dates,most_dates,years,'tas','ICHEC-EC-EARTH','rcp85','r12i1p1','SMHI-RCA4','v1','2006')
ds = xr.Dataset(
        data_vars=dict(
            tas_patterns=(["obs","time", "lat", "lon"], tas1['tas']),
            time_bnds=(["time", "bnds"], tas['time_bnds'])
        ),
        coords=dict(
            lon=(['lon'], tas['lon']),
            lat=(["lat"], tas['lat']),
            time=tas['time'],
        ),
        attrs=dict(description="Temperature patterns for" + str(year_begin)+ '-' + str(year_end)))

# Save to NCDF4
ds.to_netcdf(str(year_begin)+ '-' + str(year_end) + "_tas_patterns.nc")

ValueError: conflicting sizes for dimension 'time': length 60 on 'time' and length 365 on 'tas_patterns'

Test for patterns2tas function

In [11]:
date='20110101-20151231'
daily_climrisk_tas=np.zeros((1826,90,134))
tas_percentile=50
climrisk_tas = scipy.io.loadmat('CLIMRISK_RCP2.6_SSP1_IIASA_50pctl_50climsens.mat')['TEMPERATURES_FINAL'] # Load CLIMRISK annual temperatures 
patterns_full_path=date + '_tas_patterns.nc'
patterns_dataset = xr.open_dataset(patterns_full_path)
count=0
daily_climrisk_tas=patterns2tas(patterns_dataset,date,climrisk_tas,tas_percentile)
#for lat in range(90): #90
#    for lon in range(134): #134
#        daily_climrisk_tas[:,lat,lon]=patterns2tas(patterns_dataset,date,climrisk_tas,tas_percentile,lat,lon)
#        count+=1
#        print(str(round(count*100/(90*134),2)) + '% complete')

TypeError: patterns2tas() missing 2 required positional arguments: 'lat' and 'lon'